## 1. Cargar tuits

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
sample_rate = 0.03
chunk_size = 10000  # Adjust the chunk size as needed
all_chunks = []

for i in range(1,8):
    chunks = pd.read_csv(f'data/tweets_batch_00{i}.tsv', sep='\t',
                         usecols=["tweet_id", "user_id", "full_text"],
                         engine='python',
                         on_bad_lines='skip',
                         skiprows=lambda i: i>0 and random.random() > sample_rate,
                         chunksize=chunk_size)
    all_chunks += chunks

df = pd.concat(all_chunks, ignore_index=True)

for f in ["user_id"]:
    df[f] = df[f].astype("Int64")

In [3]:
df_full = df

In [4]:
df_full.columns

Index(['tweet_id', 'user_id', 'full_text'], dtype='object')

In [5]:
df_full.shape

(185009, 3)

In [6]:
# df = df_full.sample(10000)
df = df_full

In [7]:
len(df)

184490

In [8]:
df.to_csv("data/covid_twitter_sample_topic_model.csv", index=False)

In [9]:
docs = df.full_text.dropna().values.tolist()

In [10]:
docs[:2]

['#Covid19Mexico ha puesto en evidencia múltiples situaciones de #desigualdades en todo el mundo.\n\n#Artículo\n\n“Madre con COVID-19 enfrenta el reto de cuidar a su hijo con discapacidad” por @iartetam en @Pajaropolitico\n\n👉 https://t.co/Rzx9IqsNQd \n\n#DesigualdadesColmex\n@sedes_colmex',
 '#Reporte 🦠 Confirmamos la recuperación de 17 pacientes con coronavirus, 236 nuevos contagios y 20 fallecidos. Así, el total de personas recuperadas llega a 214, los pacientes con COVID-19 a 2.709 y 100 las víctimas. Más detalles: https://t.co/SiKTpTleFm https://t.co/XtjnGGhl9u']

In [11]:
docs[0]

'#Covid19Mexico ha puesto en evidencia múltiples situaciones de #desigualdades en todo el mundo.\n\n#Artículo\n\n“Madre con COVID-19 enfrenta el reto de cuidar a su hijo con discapacidad” por @iartetam en @Pajaropolitico\n\n👉 https://t.co/Rzx9IqsNQd \n\n#DesigualdadesColmex\n@sedes_colmex'

3. Topic modeling

## Hashtags populares

In [12]:
import re

import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return str(only_ascii)

def extract_hashtags(s):
    return [
        remove_accents(ht.lower())
        for ht in re.findall(r"#(\w+)", s)
    ]

In [13]:
df.head()

,tweet_id,user_id,full_text
0,1249125075128848384,859443522029592576,#Covid19Mexico ha puesto en evidencia múltiple...
1,1249125076424949760,116892396,#Reporte 🦠 Confirmamos la recuperación de 17 p...
2,1249125076835983362,1095458060041039872,¡Atención!\n\nLa compañía de lucha libre asegu...
3,1249125076663971840,219686511,Si ya consultaste que en tu hogar nadie ha sid...
4,1249125079042215937,433573328,"Aquí, otro héroe sin medallas."


In [14]:
df["full_text"] = df.full_text.astype(str)

In [15]:
df["hashtags"] = df.full_text.apply(extract_hashtags)

In [16]:
df.head()

,tweet_id,user_id,full_text,hashtags
0,1249125075128848384,859443522029592576,#Covid19Mexico ha puesto en evidencia múltiple...,"[b'covid19mexico', b'desigualdades', b'articul..."
1,1249125076424949760,116892396,#Reporte 🦠 Confirmamos la recuperación de 17 p...,[b'reporte']
2,1249125076835983362,1095458060041039872,¡Atención!\n\nLa compañía de lucha libre asegu...,[]
3,1249125076663971840,219686511,Si ya consultaste que en tu hogar nadie ha sid...,[b'alertacovid19sv']
4,1249125079042215937,433573328,"Aquí, otro héroe sin medallas.",[]


In [17]:
from collections import Counter

In [18]:
ht_counts = Counter()

In [19]:
for hts in df.hashtags.values:
    ht_counts.update(hts)

In [20]:
ht_counts.most_common(100)

[("b'covid19'", 56199),
 ("b'quedateencasa'", 8755),
 ("b'coronavirus'", 7007),
 ("b'covid'", 3299),
 ("b'cuba'", 2834),
 ("b'covid_19'", 1828),
 ("b'atencionmedicadecalidad'", 1309),
 ("b'venezuela'", 1199),
 ("b'cubasalvavidas'", 1196),
 ("b'estevirusloparamosunidos'", 1151),
 ("b'eeuu'", 982),
 ("b'yomequedoencasa'", 951),
 ("b'yoapoyolacuarentena'", 905),
 ("b'laprevencioneslaclave'", 868),
 ("b'covid19mx'", 828),
 ("b'venezuelabellaenrevolucion'", 800),
 ("b'ultimahora'", 792),
 ("b'17abr'", 777),
 ("b'cuarentena'", 730),
 ("b'concienciaycompromiso'", 724),
 ("b'mexico'", 670),
 ("b'cubaporlasalud'", 650),
 ("b'17anosbarrioadentro'", 611),
 ("b'abrildeunioncivicomilitar'", 565),
 ("b'abrildevictoriapopular'", 543),
 ("b'reporte'", 534),
 ("b'pandemia'", 524),
 ("b'urgente'", 520),
 ("b'ahora'", 514),
 ("b'sanadistancia'", 512),
 ("b'covid2019'", 511),
 ("b'covid19chile'", 497),
 ("b'15abr'", 496),
 ("b'atencion'", 495),
 ("b'envivo'", 492),
 ("b'envideo'", 476),
 ("b'18abr'", 466)

In [21]:
len(df)

184490

# BERTopic

In [22]:
from sentence_transformers import SentenceTransformer

In [23]:
# Spanish multilingual model (lightweight)
# This takes ages!
# model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [25]:
embeddings = model.encode(docs, batch_size=128, show_progress_bar=True)

Batches:   0%|          | 0/1442 [00:00<?, ?it/s]

In [26]:
import numpy as np
np.save("data/covid_twitter_sample_topic_model_embeddings.npy", embeddings)

In [27]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired

# Create your representation model
representation_model = KeyBERTInspired()

In [28]:
docs_sample = docs

In [29]:
embeddings_sample = embeddings

In [30]:
embeddings_sample.shape

(184484, 384)

In [31]:
# Create and fit the BERTopic model using the precomputed embeddings
topic_model = BERTopic(language="spanish")  # or language="spanish", but multilingual works well

# topics, _ = topic_model.fit_transform(docs, embeddings)
topics, _ = topic_model.fit_transform(docs_sample, embeddings_sample)

# View topic info
topic_info = topic_model.get_topic_info()


In [32]:
topic_info.head(20)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,64278,-1_te_nos_hay_lo,"[te, nos, hay, lo, todo, quedateencasa, las, m...",[Lamento profundamente la decisión de EE.UU de...
1,0,1386,0_pandemia_pandemias_encrudece_gasolinaalsecto...,"[pandemia, pandemias, encrudece, gasolinaalsec...",[🦠 La Pandemia por COVID-19 llega en el moment...
2,1,1224,1_concientizador_peda_saldría_grabar,"[concientizador, peda, saldría, grabar, vacaci...",[Ayer fue mi primer guardia en un área COVID e...
3,2,670,2_wf73icot24_sobrecogedor_2chr55juxs_0qxqormqfu,"[wf73icot24, sobrecogedor, 2chr55juxs, 0qxqorm...",[Sobrecogedor este reportaje que ha escrito @d...
4,3,616,3_récord_muertes_2371_murieron,"[récord, muertes, 2371, murieron, fallecidos, ...",[[Actualización 🇵🇪 - Abril 16] Trayectorias de...
5,4,444,4_dc9z8aaqqg_tictac_colapsada_emiten,"[dc9z8aaqqg, tictac, colapsada, emiten, monotr...",[#DolarA100\n#TicTac \n\n🚨 EL DOLAR A 100 $ \n...
6,5,434,5_alimentos_alimentaria_alimentarios_alimenticios,"[alimentos, alimentaria, alimentarios, aliment...",[Qué pueden hacer los países ante la pandemia ...
7,6,392,6_presos_cárceles_cárcel_prisión,"[presos, cárceles, cárcel, prisión, villavicen...","[#NoticiasUno| Tal como se temía, las cárceles..."
8,7,346,7_españoles_españa_arrima_español,"[españoles, españa, arrima, español, pactosdel...",[Hoy más que nunca siéntete orgulloso de ser E...
9,8,343,8_yf0shxazru_rescataron_300k_huesitos,"[yf0shxazru, rescataron, 300k, huesitos, subid...",[Chocolates valor ha subido un 20% el sueldo a...


In [33]:
import spacy
import string

# Load the Spanish library from SpaCy
nlp = spacy.load("es_core_news_sm")

# Create list of punctuation marks
punctuations = string.punctuation

# Create list of stopwords from spaCy
es_stopwords = spacy.lang.es.stop_words.STOP_WORDS

stopwords = list(es_stopwords) + ["https", "http", "com", "covid", "covid19", "19", "co", "coronavirus", "rt"]

In [34]:
len(docs_sample)

184484

In [35]:
docs_sample

['#Covid19Mexico ha puesto en evidencia múltiples situaciones de #desigualdades en todo el mundo.\n\n#Artículo\n\n“Madre con COVID-19 enfrenta el reto de cuidar a su hijo con discapacidad” por @iartetam en @Pajaropolitico\n\n👉 https://t.co/Rzx9IqsNQd \n\n#DesigualdadesColmex\n@sedes_colmex',
 '#Reporte 🦠 Confirmamos la recuperación de 17 pacientes con coronavirus, 236 nuevos contagios y 20 fallecidos. Así, el total de personas recuperadas llega a 214, los pacientes con COVID-19 a 2.709 y 100 las víctimas. Más detalles: https://t.co/SiKTpTleFm https://t.co/XtjnGGhl9u',
 '¡Atención!\n\nLa compañía de lucha libre aseguró que el trabajador viene recibiendo el tratamiento adecuado y su estado de salud es estable. \n\nhttps://t.co/FASNlvacXY',
 'Si ya consultaste que en tu hogar nadie ha sido beneficiado con el bono familiar para alimentación, completa este formulario: https://t.co/6N8k0OIJoQ.\n\nComparte este enlace con tus seres queridos y recuérdales que si no es necesario, no salgan de s

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

# Fine-tune topic representations after training BERTopic
vectorizer_model = CountVectorizer(stop_words=stopwords, ngram_range=(1, 3), min_df=10)
topic_model.update_topics(docs_sample, vectorizer_model=vectorizer_model)
info = topic_model.get_topic_info()

In [37]:
# Method 1 - safetensors
embedding_model = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
topic_model.save("covid_twitter_topic_model", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)


In [38]:
topic_model_loaded = BERTopic.load("covid_twitter_topic_model")

In [39]:
topic_model_loaded.get_topic_info().head(20)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,64278,-1_tipo_dezurdateam_diazcanelb_prevención,"[tipo, dezurdateam, diazcanelb, prevención, en...",NaN
1,0,1386,0_pandemias_pandemia_momento difícil_calidad vida,"[pandemias, pandemia, momento difícil, calidad...",NaN
2,1,1224,1_grabar_vacaciones_siento_mar,"[grabar, vacaciones, siento, mar, podido, merc...",NaN
3,2,670,2_lt_beatrizgmuller_sextanochetv_tremendo,"[lt, beatrizgmuller, sextanochetv, tremendo, p...",NaN
4,3,616,3_récord_número fallecidos_500 muertos_murieron,"[récord, número fallecidos, 500 muertos, murie...",NaN
5,4,444,4_contaminados_vice_maria_pasara,"[contaminados, vice, maria, pasara, mayoria, c...",NaN
6,5,434,5_alimentaria_alimentos_comidas_comida,"[alimentaria, alimentos, comidas, comida, alim...",NaN
7,6,392,6_presos_prisión_cárceles_cárcel,"[presos, prisión, cárceles, cárcel, presos pol...",NaN
8,7,346,7_gobierno españa_españoles_gestión gobierno_s...,"[gobierno españa, españoles, gestión gobierno,...",NaN
9,8,343,8_subido_tenerlo_plantilla_donado,"[subido, tenerlo, plantilla, donado, próxima, ...",NaN


In [40]:
info[info.Count > 100]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,64278,-1_tipo_dezurdateam_diazcanelb_prevención,"[tipo, dezurdateam, diazcanelb, prevención, en...",[Lamento profundamente la decisión de EE.UU de...
1,0,1386,0_pandemias_pandemia_momento difícil_calidad vida,"[pandemias, pandemia, momento difícil, calidad...",[🦠 La Pandemia por COVID-19 llega en el moment...
2,1,1224,1_grabar_vacaciones_siento_mar,"[grabar, vacaciones, siento, mar, podido, merc...",[Ayer fue mi primer guardia en un área COVID e...
3,2,670,2_lt_beatrizgmuller_sextanochetv_tremendo,"[lt, beatrizgmuller, sextanochetv, tremendo, p...",[Sobrecogedor este reportaje que ha escrito @d...
4,3,616,3_récord_número fallecidos_500 muertos_murieron,"[récord, número fallecidos, 500 muertos, murie...",[[Actualización 🇵🇪 - Abril 16] Trayectorias de...
...,...,...,...,...,...
118,117,105,117_morgues_fosas_tasa mortalidad_363,"[morgues, fosas, tasa mortalidad, 363, rtve, e...",[Tasa Mortalidad EEUU ➝ 62 por millón\nTasa Mo...
119,118,102,118_opacidad_publican_vox_congreso_millonarios,"[opacidad, publican, vox_congreso, millonarios...",[Algunos medios publican pagos millonarios por...
120,119,102,119_país muertos_españa país_media_española,"[país muertos, españa país, media, española, c...","[Cierto, España es el país con más muertos por..."
121,120,102,120_131_98_105_126,"[131, 98, 105, 126, contagios fallecidos, conf...",[#Reporte 🦠 Confirmamos la recuperación de 98 ...


In [41]:
res = topic_model.get_document_info(docs_sample).query("Topic != -1 & Representative_document")[['Document', 'Topic']]

In [42]:
def save_examples(n_topic):
    topic_name = info.Name[info.Topic == n_topic].values[0]
    examples = get_examples(n_topic)
    print(topic_name)
    examples.to_csv(f"examples/{topic_name}.csv", index=False)

In [43]:
%%sh
mkdir -p examples

In [45]:
info.head(100)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,64278,-1_tipo_dezurdateam_diazcanelb_prevención,"[tipo, dezurdateam, diazcanelb, prevención, en...",[Lamento profundamente la decisión de EE.UU de...
1,0,1386,0_pandemias_pandemia_momento difícil_calidad vida,"[pandemias, pandemia, momento difícil, calidad...",[🦠 La Pandemia por COVID-19 llega en el moment...
2,1,1224,1_grabar_vacaciones_siento_mar,"[grabar, vacaciones, siento, mar, podido, merc...",[Ayer fue mi primer guardia en un área COVID e...
3,2,670,2_lt_beatrizgmuller_sextanochetv_tremendo,"[lt, beatrizgmuller, sextanochetv, tremendo, p...",[Sobrecogedor este reportaje que ha escrito @d...
4,3,616,3_récord_número fallecidos_500 muertos_murieron,"[récord, número fallecidos, 500 muertos, murie...",[[Actualización 🇵🇪 - Abril 16] Trayectorias de...
...,...,...,...,...,...
95,94,113,94_quedan_aplicar_querido_única,"[quedan, aplicar, querido, única, reducir, sal...","[No hemos querido alarmar a la población, sino..."
96,95,112,95_youtuber_peligro contagio_soydavidshow_código,"[youtuber, peligro contagio, soydavidshow, cód...","[Oiga, doctor @HLGatell, si un youtuber se sal..."
97,96,112,96_capitalismo_demostró_funciona_caer,"[capitalismo, demostró, funciona, caer, moment...",[- “La pandemia demostró que el capitalismo no...
98,97,111,97_pruebas despistaje_299_mil pruebas_225,"[pruebas despistaje, 299, mil pruebas, 225, re...","[En el día de hoy 17 de abril, uno de esos hué..."


In [46]:
info.to_csv("examples/topics.csv", index=False)

In [47]:
save_examples(36)

NameError: name 'get_examples' is not defined

In [ ]:
save_examples(34)

In [ ]:
save_examples(32)

In [48]:
save_examples(12)

NameError: name 'get_examples' is not defined

In [71]:
save_examples(2)

2_educación_escolar_estudiantes_curso


In [70]:
save_examples(1)

1_pandemia_pandemia covidー19_frente pandemia_pandemia país


In [69]:
save_examples(0)

0_grabar_vacaciones_siento_mar


In [ ]:
Probability

In [59]:
get_examples(12)

,Document,Representative_document,Probability
76046,"""Quería ser útil"": La primera persona en proba...",True,1.000000
79805,"""Quería ser útil"": La primera persona en proba...",True,1.000000
80179,"""Quería ser útil"": La primera persona en proba...",True,1.000000
81217,"""Quería ser útil"": La primera persona en proba...",True,1.000000
91474,"""Quería ser útil"": La primera persona en proba...",True,1.000000
...,...,...,...
52417,La Fundación de #BillGates y su esposa está fi...,False,0.586509
100764,"3) , aluminio y mercurio ... por tejido animal...",False,0.584316
93138,🌍 😷 💉\nLa OMS anuncia que se han iniciado los ...,False,0.583863
121115,El director de la Organización Mundial de la S...,False,0.583319


In [51]:
topic_model.get_document_info(docs_sample)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,App de COVID-19 pudo ser un mero sitio web inf...,-1,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...,yomequedoencasa - tiempos - covid_19 - prevenc...,0.000000,False
1,Desafíos y oportunidades por el COVID-19 https...,-1,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...,yomequedoencasa - tiempos - covid_19 - prevenc...,0.000000,False
2,Ganamos el ultimo UHC del evento Benefico cont...,2711,2711_1000_chicos_surgió_euros,"[1000, chicos, surgió, euros, concurso, ganamo...",[Muchas gracias chicos de esta mamarrachada de...,1000 - chicos - surgió - euros - concurso - ga...,1.000000,False
3,"En toda circunstancia, en todo lugar, siempre ...",-1,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...,yomequedoencasa - tiempos - covid_19 - prevenc...,0.000000,False
4,El epidemiólogo Abdel Rodriguez Rivera habla c...,683,683_lorenzo mendoza_mendoza_escoltas_lorenzo,"[lorenzo mendoza, mendoza, escoltas, lorenzo, ...",[¡ERA DE ESPERARSE! Lorenzo Mendoza podría ten...,lorenzo mendoza - mendoza - escoltas - lorenzo...,0.400433,False
...,...,...,...,...,...,...,...,...
184582,#SociedadPA Anoche salió del Aeropuerto Intern...,-1,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...,yomequedoencasa - tiempos - covid_19 - prevenc...,0.000000,False
184583,Presidente Maduro reporta siete nuevos casos d...,319,319_nicolás maduro_nicolás_maduro_reporta casos,"[nicolás maduro, nicolás, maduro, reporta caso...",[Nicolás Maduro reporta 4 nuevos casos positiv...,nicolás maduro - nicolás - maduro - reporta ca...,0.317041,False
184584,¿Por qué al hacer compra millonaria de pruebas...,1328,1328_expresamos_generan_correspondientes_solicitó,"[expresamos, generan, correspondientes, solici...",[¿Por qué al hacer compra millonaria de prueba...,expresamos - generan - correspondientes - soli...,0.713132,True
184585,Todos coludos o todos rabones.#GobiernoDeMentiras,-1,-1_yomequedoencasa_tiempos_covid_19_prevención,"[yomequedoencasa, tiempos, covid_19, prevenció...",[Pregunta seria para quien sepa dar una respue...,yomequedoencasa - tiempos - covid_19 - prevenc...,0.000000,False


In [54]:
def get_examples(n_topic):
    res = topic_model.get_document_info(docs_sample).query(f"Topic == {n_topic}")[['Document', 'Representative_document', 'Probability']]
    return res.sort_values(['Representative_document', 'Probability'], ascending=False)

In [56]:
df = get_examples(2)

94406     La ministra de @educaciongob ha señalado que n...
106668    La ministra de @educaciongob ha señalado que n...
112329    La ministra de @educaciongob ha señalado que n...
116172    La ministra de @educaciongob ha señalado que n...
107330             ¿Qué sería de nosotros sin la educación?
                                ...                        
123463    “Extraño la rutina de mi escuela, pero sé que ...
147647    Mi señora está haciendo una maestría en derech...
91312     🗣 "Ningún alumno perderá el curso escolar por ...
21392     #NuevaYork anuncia el cierre de colegios públi...
156746    #CadaFamiliaUnaEscuela\n#16Abr | Estudiante de...
Name: Document, Length: 541, dtype: object

In [ ]:
info.Representative_Docs[3]

In [ ]:
info.shape